In [ ]:
# Scratch notebook to test rccl-analysis code

In [ ]:
# import os
# print("Current directory:", os.getcwd())
# print("Files in current directory:", os.listdir('.'))
# print("Looking for jax_rccl_analyser.py:", os.path.exists('jax_rccl_analyser.py'))

In [ ]:
import sys
sys.path.append('/workspace/code/TraceLens/TraceLens/JAXRcclAnayser')
from jax_rccl_analyser import RcclAnalyser

In [ ]:
traces_dir = "/workspace/code/traces/logs/20250509_1604_mixtral"
rank_to_pb_file_mapping = {}
world_size= 16
rank_to_pb_file_mapping["0"] = "/workspace/code/traces/logs/20250509_1604_mixtral/tw026/20250509_1604_mixtral/tensorboard/plugins/profile/2025_05_09_16_09_12/tw026.xplane.pb"
rank_to_pb_file_mapping["1"] = "/workspace/code/traces/logs/20250509_1604_mixtral/tw034/20250509_1604_mixtral/tensorboard/plugins/profile/2025_05_09_16_09_12/tw034.xplane.pb"
# rank_to_pb_file_mapping["2"] = "/workspace/code/traces/logs/20250509_1604_mixtral/tw035/20250509_1604_mixtral/tensorboard/plugins/profile/2025_05_09_16_09_12/tw035.xplane.pb"
# rank_to_pb_file_mapping["3"] = "/workspace/code/traces/logs/20250509_1604_mixtral/tw052/20250509_1604_mixtral/tensorboard/plugins/profile/2025_05_09_16_09_12/tw052.xplane.pb"

In [ ]:
rccl_analyser = RcclAnalyser(traces_dir,rank_to_pb_file_mapping,world_size)

In [ ]:
df = rccl_analyser.build_df_long()

In [ ]:
all_gather_28_df = df[df["collective_name"] == "all-gather-start.28"]

In [ ]:
all_gather_28_df

In [ ]:
# Filter DataFrame based on gpu_rank and pid
# Example: Filter for gpu_rank = 0 and pid = 1
filtered_df = all_gather_28_df[(all_gather_28_df['gpu_rank'] == 0) & (all_gather_28_df['pid'] == 1)]
# Sort by timestamp (ts) column
filtered_df = filtered_df.sort_values('ts').copy()
# Convert timestamps from nanoseconds to microseconds
filtered_df['ts_us'] = filtered_df['ts'] / 1000

# Reorder columns to have ts_us next to ts
cols = filtered_df.columns.tolist()
ts_index = cols.index('ts')
cols.insert(ts_index + 1, cols.pop(cols.index('ts_us')))
filtered_df = filtered_df[cols]

print(f"Original DataFrame shape: {all_gather_28_df.shape}")
print(f"Filtered DataFrame shape: {filtered_df.shape}")
filtered_df

In [ ]:
# More filtering examples:

# 1. Filter for specific gpu_rank only
gpu_rank_0_df = all_gather_28_df[all_gather_28_df['gpu_rank'] == 0]
print(f"Events for gpu_rank 0: {gpu_rank_0_df.shape[0]} events")

# 2. Filter for specific pid only
pid_1_df = all_gather_28_df[all_gather_28_df['pid'] == 1]
print(f"Events for pid 1: {pid_1_df.shape[0]} events")

# 3. Filter for multiple gpu_ranks
multi_gpu_ranks_df = all_gather_28_df[all_gather_28_df['gpu_rank'].isin([0, 15])]
print(f"Events for gpu_ranks 0 and 15: {multi_gpu_ranks_df.shape[0]} events")

# 4. Show unique values for reference
print(f"\nUnique gpu_ranks: {sorted(all_gather_28_df['gpu_rank'].unique())}")
print(f"Unique pids: {sorted(all_gather_28_df['pid'].unique())}")

# 5. Group by gpu_rank and pid to see distribution
distribution = all_gather_28_df.groupby(['gpu_rank', 'pid']).size().reset_index(name='count')
print(f"\nDistribution of events by gpu_rank and pid:")
distribution

In [ ]:
df = rccl_analyser.build_df_nccl_implicit_sync_cat()

In [ ]:
df.

In [ ]:
rccl_analyser.df_implicit_sync_cat_detailed.head()

In [ ]:
rccl_analyser.df_implicit_sync_cat_detailed[rccl_analyser.df_implicit_sync_cat_detailed["collective_name"] == "all-reduce-start"]

In [ ]:
sum_comm_latency = rccl_analyser.df_implicit_sync_cat_detailed[rccl_analyser.df_implicit_sync_cat_detailed["collective_name"] == "all-gather-start.197"]["comm_latency"].sum()

In [ ]:
sum_comm_latency*1e-9

In [ ]:
rccl_analyser.collectives_size_map["all-reduce-start"][1]

In [ ]:
rccl_analyser.collectives_size_map["all-gather-start.197"]

In [ ]:
201326592/(1024*1024)


In [ ]:
192/2.705485e-05

In [ ]:
import re

# Get unique collective names
collective_names = df['collective_name'].unique()

# Extract operation names without numbers
def extract_base_operation_name(name):
    # Remove numbers and dots at the end
    base_name = re.sub(r'\.\d+.*$', '', name)
    # Remove standalone numbers at the end
    base_name = re.sub(r'\.\d+$', '', base_name)
    return base_name

# Get unique base operation names
base_operations = set()
for name in collective_names:
    base_name = extract_base_operation_name(name)
    base_operations.add(base_name)

# Sort for consistent output
unique_base_operations = sorted(list(base_operations))
print("Unique collective operation types:")
for op in unique_base_operations:
    print(f"  - {op}")

In [ ]:
rccl_analyser.collectives_size_map["all-gather-start.197"][2]

In [ ]:
df[df["collective_name"] == "all-gather-start.197"]

In [ ]:
rank_events = df[df["collective_id"] == "all-gather-start.197_2"]

In [ ]:
rank_events.reset_index(drop=True)